In [ ]:
import json
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Set up the WebDriver (Chrome example)
driver = webdriver.Chrome()  # Make sure chromedriver is in PATH

# Open a webpage
url = 'https://www.imdb.com/title/tt13918776'

driver.get(url)
time.sleep(5)

def get_casts():
  section = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[5]/div[2]/div[2]')
  casts = section.find_elements(By.XPATH, './div')
  print(len(casts))
  res = []
  for cast in casts:
    cast_avatar = cast.find_element(By.TAG_NAME, 'img').get_attribute('src')
    el = cast.find_elements(By.XPATH, './div')[1].find_elements(By.TAG_NAME, 'a')
    cast_name = el[0].text.strip()
    cast_role = el[1].text.strip()
    res.append({
      'name': cast_name,
      'role': cast_role,
      'avatar': cast_avatar
    })

  with open('cast.json', 'w') as f:
    json.dump(res, f, indent=2)
  
try:
  el = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[1]/a'))
  )
  driver.execute_script("arguments[0].scrollIntoView(true);", el)
  time.sleep(1)  # Give the page time to settle after scrolling
  el.click()
  time.sleep(5)
  seasons = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[1]/div[2]/ul'))
  )
  driver.execute_script("arguments[0].scrollIntoView(true);", seasons)
  seasons = seasons.find_elements(By.XPATH, './a')
  seasons = [season.text for season in seasons]
  photo_album = {}
  for season in seasons:
    new_url = url + f'/episodes?season={season}&ref_=ttep'
    driver.get(new_url)
    wrapper = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]'))
    )
    episode_section = wrapper.find_elements(By.XPATH, './article')
    episode_hrefs = [episode.find_element(By.TAG_NAME, 'a').get_attribute('href') for episode in episode_section]
    for index, episode_href in enumerate(episode_hrefs):
      # episode_href = episode.find_element(By.TAG_NAME, 'a').get_attribute('href')
      driver.get(episode_href)
      print(f'S{season}.E{index + 1} {episode_href}')
      photo_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div'))
      )
      if len(photo_button.find_elements(By.CSS_SELECTOR, 'span.ipc-title__subtext')) > 0:
        photo_button = photo_button.find_element(By.TAG_NAME, 'a')
        driver.get(photo_button.get_attribute('href'))
        photo_grid_button = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div[3]/a'))
        )
      
        driver.get(photo_grid_button.get_attribute('href'))
        photo_section = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[1]/section/div'))
        )
        photos = photo_section.find_elements(By.TAG_NAME, 'img')
        photo_album[f'S{season}.E{index + 1}'] = [photo.get_attribute('src') for photo in photos]

  with open('photo_album.json', 'w') as f:
    json.dump(photo_album, f, indent=2)


except Exception as e:
  print(e)
finally:
  driver.quit()


In [2]:
import json
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=tv_series'
driver.get(url)
ul = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul'))
)
movie_cards = ul.find_elements(By.XPATH, './li')
movie_hrefs = []
for page in tqdm(range(1, 700)):
  try:
    load_more_button = WebDriverWait(driver, 10).until(
      EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button'))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", load_more_button)
    ul = WebDriverWait(driver, 180).until(
      EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul'))
    )
    movie_cards = ul.find_elements(By.XPATH, './li')
    if page > 1:
      for previous_movie_card in movie_cards[:50]:
        driver.execute_script("arguments[0].remove();", previous_movie_card)
    movie_cards = ul.find_elements(By.XPATH, './li')
    print(f'page {page} {len(movie_cards)}')
    movie_page_hrefs = [movie_card.find_element(By.TAG_NAME, 'a').get_attribute('href') for movie_card in movie_cards]
    with open('test.jsonl', 'a') as f:
      f.write(json.dumps(movie_page_hrefs) + '\n')
  except Exception as e:
    print(e)
    break




 14%|█▎        | 95/699 [06:08<39:01,  3.88s/it]


KeyboardInterrupt: 